In [2]:
import gradio as gr
import os
import shutil
from PIL import Image

# ==============================================================================
# [1. 경로 및 설정]
# ==============================================================================

GENERATED_DIR = r"C:\Users\nam\Desktop\output_test"
REF_C_ROOT_DIR = r"C:\Users\nam\Desktop\TTF_files\fontdataset_output"
REVIEW_BASE_DIR = r"C:\Users\nam\Desktop\review_result_classified"

# 변경된 폴더명 설정 (키값이 실제 폴더명이 됩니다)
LABEL_MAP = {
    "SS": "1. 폰트O 글자O (Same/Same)",
    "DS": "2. 폰트X 글자O (Diff/Same)",
    "SD": "3. 폰트O 글자X (Same/Diff)",
    "DD": "4. 폰트X 글자X (Diff/Diff)",
    "Trash": "🗑️ 버리기 (Trash)"
}

# ==============================================================================
# [2. 유틸리티 및 로직 함수]
# ==============================================================================

def ensure_dirs():
    """결과 저장용 디렉토리 생성"""
    for folder_name in LABEL_MAP.keys():
        os.makedirs(os.path.join(REVIEW_BASE_DIR, folder_name), exist_ok=True)

def get_generated_files():
    """생성된 이미지 파일 목록 로드"""
    if not os.path.exists(GENERATED_DIR):
        print(f"⚠️ 경로를 찾을 수 없음: {GENERATED_DIR}")
        return []
    files = [f for f in os.listdir(GENERATED_DIR) if f.lower().endswith('.png') and f.startswith("TEST_")]
    return sorted(files)

def parse_filename_and_get_ref(gen_filename):
    """파일명 파싱 및 원본 이미지 경로 추적"""
    try:
        name_without_ext = os.path.splitext(gen_filename)[0]
        parts = name_without_ext.split('_')

        if len(parts) < 3:
            return None, None

        file_suffix_parts = parts[-2:]
        target_filename = "_".join(file_suffix_parts) + ".png"

        candidate_parts = parts[:-2]
        found_folder = None
        current_name_check = ""

        for i in range(len(candidate_parts) - 1, -1, -1):
            part = candidate_parts[i]
            if current_name_check == "":
                current_name_check = part
            else:
                current_name_check = part + "_" + current_name_check

            path_underscore = os.path.join(REF_C_ROOT_DIR, current_name_check)
            name_space = current_name_check.replace("_", " ")
            path_space = os.path.join(REF_C_ROOT_DIR, name_space)

            if os.path.isdir(path_underscore):
                found_folder = current_name_check
                break
            elif os.path.isdir(path_space):
                found_folder = name_space
                break

        if found_folder is None:
            return None, None

        ref_image_path = os.path.join(REF_C_ROOT_DIR, found_folder, target_filename)

        if os.path.exists(ref_image_path):
            return ref_image_path, found_folder
        else:
            return None, found_folder

    except Exception as e:
        print(f"❌ 파싱 오류 ({gen_filename}): {e}")
        return None, None

def load_images(file_list, index):
    """이미지 로드 및 화면 갱신"""
    if not file_list or index >= len(file_list):
        return None, None, "🎉 모든 리뷰가 완료되었습니다!"

    gen_filename = file_list[index]
    gen_path = os.path.join(GENERATED_DIR, gen_filename)
    ref_path, detected_folder = parse_filename_and_get_ref(gen_filename)

    gen_img = Image.open(gen_path) if os.path.exists(gen_path) else None

    ref_img = None
    if ref_path and os.path.exists(ref_path):
        ref_img = Image.open(ref_path)

    status_text = f"[{index+1} / {len(file_list)}] 현재 파일: {gen_filename}\n"
    if detected_folder:
        status_text += f"📂 감지된 폰트: {detected_folder}"
    else:
        status_text += "⚠️ 폰트 폴더 감지 실패"

    if ref_img is None:
        status_text += " | ❌ Ref 이미지 없음"

    return gen_img, ref_img, status_text

def classify_and_move(label_key, file_list, index):
    """파일 이동 로직"""
    if not file_list or index >= len(file_list):
        return load_images(file_list, index) + (index,)

    gen_filename = file_list[index]
    src_path = os.path.join(GENERATED_DIR, gen_filename)

    dest_dir = os.path.join(REVIEW_BASE_DIR, label_key)
    dest_path = os.path.join(dest_dir, gen_filename)

    # 1. 이미지 이동
    if os.path.exists(src_path):
        try:
            if os.path.exists(dest_path):
                os.remove(dest_path)
            shutil.move(src_path, dest_path)
            print(f"[{label_key}] Moved: {gen_filename}")
        except Exception as e:
            print(f"🚨 이동 실패: {e}")

    # 2. 폰트 폴더 복사
    _, font_folder_name = parse_filename_and_get_ref(gen_filename)
    if font_folder_name:
        src_font_dir = os.path.join(REF_C_ROOT_DIR, font_folder_name)
        dest_font_dir = os.path.join(dest_dir, font_folder_name)

        if os.path.exists(src_font_dir) and not os.path.exists(dest_font_dir):
            try:
                shutil.copytree(src_font_dir, dest_font_dir)
            except Exception:
                pass

    new_index = index + 1
    return load_images(file_list, new_index) + (new_index,)

# ==============================================================================
# [3. UI 설정]
# ==============================================================================

custom_css = """
.gradio-container { max-width: 95% !important; }
#img_row { height: 70vh !important; }
#img_display { height: 100% !important; object-fit: contain; }
#status_box { font-size: 1.2em; font-weight: bold; }
"""

shortcut_js = """
<script>
document.addEventListener('keydown', function(e) {
    if (e.target.tagName === 'TEXTAREA' || e.target.tagName === 'INPUT') return;
    if (e.key === "1") document.getElementById('btn_1').click();
    else if (e.key === "2") document.getElementById('btn_2').click();
    else if (e.key === "3") document.getElementById('btn_3').click();
    else if (e.key === "4") document.getElementById('btn_4').click();
    else if (e.key === " " || e.key === "Delete" || e.key === "ArrowRight") document.getElementById('btn_trash').click();
});
</script>
"""

ensure_dirs()
initial_files = get_generated_files()

with gr.Blocks(title="Font Classifier", head=shortcut_js, css=custom_css, theme=gr.themes.Soft()) as demo:
    file_list_state = gr.State(initial_files)
    current_index_state = gr.State(0)
    status_display = gr.Textbox(show_label=False, interactive=False, text_align="center", value="Ready", elem_id="status_box")

    with gr.Row(elem_id="img_row"):
        gen_image_display = gr.Image(label="Generated (평가 대상)", type="pil", interactive=False, elem_id="img_display")
        ref_image_display = gr.Image(label="Reference (기준)", type="pil", interactive=False, elem_id="img_display")

    gr.Markdown("### 👇 키보드: [1] SS / [2] DS / [3] SD / [4] DD / [Space] Trash")

    with gr.Row():
        # 버튼 라벨은 사용자 편의를 위해 상세히 적되, 실제 전달되는 값(lambda)은 폴더명(SS, DS...)
        btn_1 = gr.Button("1. SS (폰트= 글자=)", variant="primary", elem_id="btn_1")
        btn_2 = gr.Button("2. DS (폰트≠ 글자=)", variant="secondary", elem_id="btn_2")
        btn_3 = gr.Button("3. SD (폰트= 글자≠)", variant="secondary", elem_id="btn_3")
        btn_4 = gr.Button("4. DD (폰트≠ 글자≠)", variant="secondary", elem_id="btn_4")
        btn_trash = gr.Button("🗑️ Trash", variant="stop", elem_id="btn_trash")

    common_outputs = [gen_image_display, ref_image_display, status_display, current_index_state]
    demo.load(load_images, [file_list_state, current_index_state], [gen_image_display, ref_image_display, status_display])

    # 폴더명(SS, DS, SD, DD)으로 매핑 수정
    btn_1.click(lambda f, i: classify_and_move("SS", f, i), [file_list_state, current_index_state], common_outputs)
    btn_2.click(lambda f, i: classify_and_move("DS", f, i), [file_list_state, current_index_state], common_outputs)
    btn_3.click(lambda f, i: classify_and_move("SD", f, i), [file_list_state, current_index_state], common_outputs)
    btn_4.click(lambda f, i: classify_and_move("DD", f, i), [file_list_state, current_index_state], common_outputs)
    btn_trash.click(lambda f, i: classify_and_move("Trash", f, i), [file_list_state, current_index_state], common_outputs)

if __name__ == "__main__":
    print(f"🚀 실행 중... 폴더명 확인: SS, DS, SD, DD, Trash")
    demo.launch(inbrowser=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34904\3607035440.py:177: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css, head. Please pass these parameters to launch() instead.
  with gr.Blocks(title="Font Classifier", head=shortcut_js, css=custom_css, theme=gr.themes.Soft()) as demo:


⚠️ 경로를 찾을 수 없음: C:\Users\nam\Desktop\output_test
🚀 실행 중... 폴더명 확인: SS, DS, SD, DD, Trash
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
